In [ ]:
import cv2 as cv
import numpy as np
import time
from pyModbusTCP.client import ModbusClient

In [ ]:
#Function to convert pixel coordinates to world mm coordinates
#Camera is considered to be placed at a certain height and pixel to mm conversion factors in X and Y axes 
#are calculated by mathematical calculations
def getWorldCoordinates(cameraCoordinates):
    
    x1 = cameraCoordinates[0]*0.38
    y1 = cameraCoordinates[1]*0.37
    
    print("Coordinates in mm - ", x1, y1)
    
    return (x1, y1)

In [ ]:
#Since the robot origin is different than the image origin, the following function is used to add offsets to 
#translate the image origin to the robot origin
def addRobotOffset(world_center):
    
    x2 = world_center[0]+19.6
    y2 = world_center[1]
    
    return (x2, y2)

In [ ]:
#The algorithm used for calculating the rotation angle calculates angle only in range of 0 to 180 degrees
#However, modbus registers used can only hold unsignd integer values and thus, to handle clock-wise rotations, 
#following function is used to convert angle value in range of 0 to 360
def adjustRotationAngle(angle, m5_center, keyhole_center):
    
    actual_angle = 0
    
    if angle <= 90:
        if m5_center[1] > keyhole_center[1]:
            actual_angle = 180 + angle
        else:
            actual_angle = angle
    else:
        if m5_center[1] < keyhole_center[1]:
            actual_angle = 180 + angle
        else:
            actual_angle = angle
            
    if actual_angle == 180:
        actual_angle = 0
            
    return actual_angle

In [ ]:
#Following function is used to get the rotation angle of the box
#Considering the rigid structure of the box and colour combinations, we have selected M5 button for calculating angle
#The following function converts original BGR image into HSV format and masks out all red components. 
#Then, by filtering out required area and shape, it identifies the M5 button and calculates
#its rotation angle w.r.t. the horizontal axis
def findRotationAngle(input_image):
    
    img = input_image.copy()
    
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    lower_red = np.array([0, 70, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv.inRange(hsv, lower_red, upper_red)

    lower_red = np.array([170, 70, 50])
    upper_red = np.array([180, 245, 245])
    mask2 = cv.inRange(hsv, lower_red, upper_red)

    mask = mask1 | mask2

    contour, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    cntsSorted = sorted(contour, key=lambda x: cv.contourArea(x))

    blank = img.copy()
    rotation_angle = 0
    center = (0, 0)

    for cnt in cntsSorted:
        area = cv.contourArea(cnt)
        #print(area)
        if 8000 < area < 12000:
            rect = cv.minAreaRect(cnt)
            box = cv.boxPoints(rect)
            box = np.int0(box)
            center = rect[0]
            w, h = rect[1]
            if w < h:
                rotation_angle = 90-rect[-1]
            else:
                rotation_angle = 180-rect[-1]
            cv.drawContours(img,[box],0,(0,0,255),2)
            cv.drawContours(blank, [cnt], -1, (0, 255, 0), 2)
            
    cv.imshow("contours_image", blank)
    
    return center, rotation_angle

In [ ]:
#Final keyhole is a very significant component on the box and it can be used as a reference point for all other components
#The below function identifies the keyhole using Canny edge detection algorithm and then finds the center point of the
#keyhole using moments of its contour
def findCenterCoord(img, m5_center):
    
    original = img.copy()
    gray = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray, (7, 7), 0)
    
    canny = cv.Canny(blur, 70, 255, 0)
    
    cv.imshow("Canny", canny)
    
    conts, hir = cv.findContours(canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    contsSorted = sorted(conts, key=lambda x: cv.contourArea(x))
    blank = img.copy()
    
    cx=0
    cy=0
    diff = 1000
    m5_y = m5_center[1]
    
    center_x = 0
    center_y = 0
    
    for c in contsSorted:
        area = cv.contourArea(c)
        if 1300 < area < 2000:
            M = cv.moments(c)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            if abs(cy-m5_y) < diff:
                diff = abs(cy-m5_y)
                center_x = cx
                center_y = cy
                cv.drawContours(blank, [c], -1, (255, 0, 0), 2)
                cv.circle(blank, (cx, cy), 5, (0, 0, 255), 2)
            
        
    cv.imshow("Contours", blank)
    return (center_x, center_y)

In [ ]:
#Function to set initial camera parameters
def setCamera(cam):
    
    cap = cv.VideoCapture(cam)
    cap.set(cv.CAP_PROP_SETTINGS, 1)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720)
    
    return cap

In [ ]:
#Function to configure MODBUS connection on given server and port
def configureModbus(server, port):

    client = ModbusClient()

    client.host(server)
    client.port(port)
    client.open()
        
    if client.is_open():
        print("Modbus connection successfully established")
    else:
        print("Connection not established")
    
    return client

In [ ]:
#Function to send a single value on MODBUS channel on a given address
def sendValue(client, address, val):
    val = round(val*10)
    print("Value - ", val, " sent at address - ", address)
    client.write_single_register(address, val)

In [ ]:
#Function to find undistorted image by applying the camera matrix
#Camera matrix is pre-calculated with camera position at a certain height
def getCalibratedImage(img):
    
    camera_matrix = np.array([[1.25203035e+03, 0.00000000e+00, 6.66075086e+02],
                     [0.00000000e+00, 1.25370911e+03, 3.62197565e+02],
                     [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
    
    distortion = np.array([[ 0.05018599,  0.63090903,  0.00656325,  0.00949236, -1.97458389]])
    
    h, w = img.shape[:2]
    optimal_camera_matrix, roi = cv.getOptimalNewCameraMatrix(camera_matrix, distortion, (w, h), 1, (w, h))
    
    undistorted_image = cv.undistort(img, camera_matrix, distortion, None, optimal_camera_matrix)
    
    cv.imshow("Undistorted Image", undistorted_image)
    
    return undistorted_image

In [ ]:
def main():
    
    #Set camera parameters for capturing
    cap = setCamera(0)

    #Capture 120 frames for adjusting focus
    for i in range(120):
        ret, image = cap.read()
        cv.imshow("Pre-trial", image)
        cv.waitKey(1)

    # Capture processing frame and get calibrated image by applying calibration matrix
    ret, image = cap.read()
    calibrated_image = getCalibratedImage(image)
    
    # Calculate center and rotation angle in image frame
    m5_center, rotation_angle_180 = findRotationAngle(calibrated_image)
    center = findCenterCoord(calibrated_image, m5_center)
    rotation_angle = adjustRotationAngle(rotation_angle_180, m5_center, center)
    
    # Convert center coordinates to world plane
    world_center = getWorldCoordinates(center)
    
    # Add robot plane offsets to the world coordinates
    robot_coordinates_raw = addRobotOffset(world_center)
    
    # Apply clockwise rotation to Y by 4 degree
    robot_coordinates_final = rotateYBy4(robot_coordinates_raw)
    
    #Configure modbus and send values
    client = configureModbus("194.94.86.6", 502)

    address = 24576
    print("Sending angle value - ", rotation_angle, " at address ", address)
    sendValue(client, address, rotation_angle)

    address = 24637
    print("Sending x - ", robot_coordinates_final[0], " at address ", address)
    sendValue(client, address, robot_coordinates_final[0])

    address = address + 1
    print("Sending y - ", robot_coordinates_final[1], " at address ", address)
    sendValue(client, address, robot_coordinates_final[1])

    address = address + 1
    print("Sending z - ", 130, " at address ", address)
    sendValue(client, address, 130)

    k = cv.waitKey(0)
    if k==ord("q"):
        cv.destroyAllWindows()

In [ ]:
main()